In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)



## Some background reading

###### Space Weather:
- [Introduction](https://ccmc.gsfc.nasa.gov/RoR_WWW/SWREDI/2016/SpaceWeatherIntro_Bootcamp_2016.pdf)
- [Understanding space weather](https://www.sciencedirect.com/science/article/pii/S0273117715002252)

###### Particle Precipitation:
Here are a few particle precipitation resources that I believe are most valuable to start with:
- Technical details of the observations: [Redmon et al., [2017]](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016JA023339)
- Creating particle precipitation models from these data: [Hardy et al., [1987]](https://doi.org/10.1029/JA090iA05p04229) and [Newell et al., [2009]](https://doi.org/10.1029/2009JA014326)
- Considered the 'state of the art' model: [OVATION PRIME](https://ccmc.gsfc.nasa.gov/models/modelinfo.php?model=Ovation%20Prime)



## Imports and utility functions


In [1]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import datetime
from os.path import isfile, join
from sys import getsizeof
import glob

from random import *

from sklearn import *

## Prepare data for ML exploration (read in full DB created from standard_ML_DB_preparation.ipynb)


In [2]:
file_load_df_cumulative = 'ML_DB_subsamp_ext_full_dfCumulative_complexHemisphereCombine.csv'
DMSP_DATA_DIR=''
df_cumulative = pd.read_csv(os.path.join(DMSP_DATA_DIR,file_load_df_cumulative))
df_cumulative = df_cumulative.set_index('Datetimes')
df_cumulative.index = pd.to_datetime(df_cumulative.index)

cols_to_drop_validation = [c for c in df_cumulative.columns if ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]
# cols_to_drop_validation = [c for c in df.columns if ('1min' in c) | ('3min' in c) | ('4min' in c) | ('5min' in c) | ('15min' in c) | ('newell' in c) | ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]

df_cumulative = df_cumulative.drop(columns=cols_to_drop_validation)


In [3]:
df_cumulative.shape



(1947016, 149)

In [4]:
# Separate training and testing data
mask_val = [(df_cumulative.index.year == 2010) & (df_cumulative['ID_SC'].values==16)]
df_val = df_cumulative[mask_val[0]].copy(deep=True)
df_train = df_cumulative.copy(deep=True).drop( df_cumulative.index[mask_val[0]])
print('validation data shape = {}'.format(df_val.shape))
print('train data shape = {}'.format(df_train.shape))
print('NOTE: we will use CV on the train data below to define model training and testing data,\n  so have called the withheld data *validation* data here')

# Construct X and y
feature_cols = [c for c in df_cumulative.columns if not 'ELE' in c]
#print( (feature_cols))
#print(df_cumulative.columns)

X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
#print(numFeatures)

validation data shape = (55210, 149)
train data shape = (1838283, 149)
NOTE: we will use CV on the train data below to define model training and testing data,
  so have called the withheld data *validation* data here


In [5]:
y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))

%matplotlib inline  
import matplotlib.pyplot as plt
X = np.array(X_train_scaled, dtype=np.float32)
X_test = np.array(X_val_scaled, dtype=np.float32)

Y = np.array(y_train_log, dtype=np.float32)
X.shape
X[:,2].size
Y.size

1838283

In [6]:

# plt.figure(figsize=(20,20))
# #plt.scatter(X[:,3],Y)
# plt.plot(X[:1000,2])
# plt.plot(Y[:1000])

# #plt.show()



In [7]:
X_train_scaled.shape[1]
hist_len = 128

In [ ]:
X_train_scaled_hist = np.zeros((X.shape[0], hist_len,22, 1), dtype=np.float32)
X_test_scaled_hist = np.zeros((X_test.shape[0], hist_len,22, 1), dtype=np.float32)

for i in range(hist_len,X.shape[0]):
    for j in range(22):    
        X_train_scaled_hist[i-hist_len,:,j,0]= X[i-hist_len:i,j]
        

for i in range(hist_len,X_test.shape[0]):
    for j in range(22):    
        X_test_scaled_hist[i-hist_len,:,j,0]= X_test[i-hist_len:i,j]

In [ ]:
# results = modelhist.evaluate(X_train_scaled_hist)#, y_val_log, batch_size=128)
# plt.figure(figsize=(50,15))

# plt.plot(y_train_log.values[:1000])
# plt.plot(results[:1000])
# plt.show()

# results = modelhist.evaluate(X_test_scaled_hist)#, y_val_log, batch_size=128)
# plt.figure(figsize=(50,15))

# plt.plot(y_val_log.values[:1000])
# plt.plot(results[:1000])
# plt.show()

In [16]:
import numpy as np
X_with_log = np.zeros((X.shape[0],148*2))
X_test_with_log = np.zeros((X_test.shape[0],148*2))

X_with_log[:,:148]= X
X_test_with_log[:,:148] = X_test
X_with_log[:,148:]= np.log(abs(X)+.0001)
X_test_with_log[:,148:] = np.log(abs(X_test)+.0001)

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers

model.add(Dense(int(148*2), activation='relu'))
model.add(Dense(44, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_with_log, np.array(y_train_log), validation_data=(X_test_with_log, np.array(y_val_log)), epochs=6)

Train on 1838283 samples, validate on 55210 samples
Epoch 1/6
1838283/1838283 [==============================] - 118s 64us/step - loss: 1.0459 - accuracy: 0.0000e+00 - val_loss: 0.7121 - val_accuracy: 0.0000e+00
Epoch 2/6
1838283/1838283 [==============================] - 141s 77us/step - loss: 0.5601 - accuracy: 0.0000e+00 - val_loss: 0.6695 - val_accuracy: 0.0000e+00
Epoch 3/6
1838283/1838283 [==============================] - 144s 78us/step - loss: 0.5072 - accuracy: 5.4399e-07 - val_loss: 0.6869 - val_accuracy: 0.0000e+00
Epoch 4/6
1838283/1838283 [==============================] - 121s 66us/step - loss: 0.5072 - accuracy: 0.0000e+00 - val_loss: 0.6790 - val_accuracy: 0.0000e+00
Epoch 5/6
1838283/1838283 [==============================] - 121s 66us/step - loss: 0.4893 - accuracy: 5.4399e-07 - val_loss: 0.7120 - val_accuracy: 0.0000e+00
Epoch 6/6
1838283/1838283 [==============================] - 120s 65us/step - loss: 0.4790 - accuracy: 0.0000e+00 - val_loss: 0.6498 - val_accuracy:

In [ ]:
results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)

#print(X_val_scaled, y_val_log.values, results)
plt.figure(figsize=(200,20))

plt.plot(y_val_log.values)
plt.plot(results)
plt.show()
plt.figure(figsize=(50,15))

plt.plot(y_val_log.values[:1000])
plt.plot(results[:1000])
plt.show()

In [ ]:
plt.figure(figsize=(50,15))

plt.plot(y_val_log.values[:1000])
plt.plot(results[:1000])
plt.show()

In [15]:



X_log= np.log(abs(X)+.0001)
X_test_log = np.log(abs(X_test)+.0001)


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers

model.add(Dense(int(148), activation='relu'))
model.add(Dense(22, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.fit(X_log, np.array(y_train_log), validation_data=(X_test_log, np.array(y_val_log)), epochs=40)

Train on 1838283 samples, validate on 55210 samples
Epoch 1/40
1838283/1838283 [==============================] - 132s 72us/step - loss: 1.2530 - accuracy: 0.0000e+00 - val_loss: 1.7673 - val_accuracy: 0.0000e+00
Epoch 2/40
1838283/1838283 [==============================] - 149s 81us/step - loss: 1.0863 - accuracy: 0.0000e+00 - val_loss: 1.7112 - val_accuracy: 0.0000e+00
Epoch 3/40
1838283/1838283 [==============================] - 181s 98us/step - loss: 1.0611 - accuracy: 0.0000e+00 - val_loss: 1.6446 - val_accuracy: 0.0000e+00
Epoch 4/40
1838283/1838283 [==============================] - 184s 100us/step - loss: 1.0473 - accuracy: 0.0000e+00 - val_loss: 1.6548 - val_accuracy: 0.0000e+00
Epoch 5/40
1838283/1838283 [==============================] - 198s 108us/step - loss: 1.0399 - accuracy: 0.0000e+00 - val_loss: 1.6731 - val_accuracy: 0.0000e+00
Epoch 6/40
1838283/1838283 [==============================] - 159s 86us/step - loss: 1.0345 - accuracy: 0.0000e+00 - val_loss: 1.6533 - val_a

KeyboardInterrupt: 

In [23]:

from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, Input, LSTM, TimeDistributed, MaxPooling1D
from keras.layers.merge import concatenate

In [35]:
X1=X.reshape((X.shape[0],148,1))
X_test1=X_test.reshape((X_test.shape[0],148,1))


input1 = Input(shape=(148,1))

model=Dense(int(32), activation='relu')(input1)

merged=TimeDistributed(Flatten())(model)
merged=LSTM(8,activation='relu')(merged)

output = Dense(4, activation='relu')(merged)
output = Dense(1)(output)

merged_model = Model(inputs = input1, outputs =output )
merged_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
merged_model.summary()
merged_model.fit(X1, y_train_log, validation_data=(X_test1, y_val_log), epochs=6)


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 148, 1)            0         
_________________________________________________________________
dense_58 (Dense)             (None, 148, 32)           64        
_________________________________________________________________
time_distributed_12 (TimeDis (None, 148, 32)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 8)                 1312      
_________________________________________________________________
dense_59 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 5         
Total params: 1,417
Trainable params: 1,417
Non-trainable params: 0
________________________________________________________

KeyboardInterrupt: 

In [41]:
X1=X.reshape((X.shape[0],1,148))
X_test1=X_test.reshape((X_test.shape[0],1,148))


input1 = Input(shape=(1,148))

model=Dense(int(148), activation='relu')(input1)

merged=TimeDistributed(Flatten())(input1)
merged=LSTM(32,activation='relu')(merged)

output = Dense(4, activation='relu')(merged)
output = Dense(1)(output)

merged_model = Model(inputs = input1, outputs =output )
merged_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
merged_model.summary()
merged_model.fit(X1, y_train_log, validation_data=(X_test1, y_val_log), epochs=6)


Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 1, 148)            0         
_________________________________________________________________
time_distributed_18 (TimeDis (None, 1, 148)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 32)                23168     
_________________________________________________________________
dense_77 (Dense)             (None, 4)                 132       
_________________________________________________________________
dense_78 (Dense)             (None, 1)                 5         
Total params: 23,305
Trainable params: 23,305
Non-trainable params: 0
_________________________________________________________________
Train on 1838283 samples, validate on 55210 samples
Epoch 1/6
1838283/1838283 [==============================] - 192s 10

In [42]:
merged_model.fit(X1, y_train_log, validation_data=(X_test1, y_val_log), epochs=34)


Train on 1838283 samples, validate on 55210 samples
Epoch 1/34
1838283/1838283 [==============================] - 207s 113us/step - loss: 0.4713 - accuracy: 5.4399e-07 - val_loss: 0.6881 - val_accuracy: 0.0000e+00
Epoch 2/34
1838283/1838283 [==============================] - 207s 113us/step - loss: 0.4665 - accuracy: 5.4399e-07 - val_loss: 0.6654 - val_accuracy: 0.0000e+00
Epoch 3/34
1838283/1838283 [==============================] - 209s 114us/step - loss: 0.4632 - accuracy: 5.4399e-07 - val_loss: 0.6558 - val_accuracy: 0.0000e+00
Epoch 4/34
1838283/1838283 [==============================] - 210s 114us/step - loss: 0.4601 - accuracy: 5.4399e-07 - val_loss: 0.6562 - val_accuracy: 0.0000e+00
Epoch 5/34
1838283/1838283 [==============================] - 210s 114us/step - loss: 0.4605 - accuracy: 5.4399e-07 - val_loss: 0.6415 - val_accuracy: 0.0000e+00
Epoch 6/34
1838283/1838283 [==============================] - 195s 106us/step - loss: 0.4577 - accuracy: 5.4399e-07 - val_loss: 0.6376 - v

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()model
#add model layers

model.add(Dense(int(32), activation='relu'))
model.add(Dense(22, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.fit(X, np.array(y_train_log), validation_data=(X_test, np.array(y_val_log)), epochs=40)

Train on 1838283 samples, validate on 55210 samples
Epoch 1/40
1838283/1838283 [==============================] - 118s 64us/step - loss: 1.4170 - accuracy: 0.0000e+00 - val_loss: 0.8645 - val_accuracy: 0.0000e+00
Epoch 2/40
1838283/1838283 [==============================] - 157s 86us/step - loss: 0.5446 - accuracy: 5.4399e-07 - val_loss: 0.7546 - val_accuracy: 0.0000e+00
Epoch 3/40
1838283/1838283 [==============================] - 153s 83us/step - loss: 0.4959 - accuracy: 5.4399e-07 - val_loss: 0.6570 - val_accuracy: 0.0000e+00
Epoch 4/40
1838283/1838283 [==============================] - 133s 72us/step - loss: 0.4773 - accuracy: 5.4399e-07 - val_loss: 0.6774 - val_accuracy: 0.0000e+00
Epoch 5/40
1838283/1838283 [==============================] - 138s 75us/step - loss: 0.4762 - accuracy: 5.4399e-07 - val_loss: 0.6612 - val_accuracy: 0.0000e+00
Epoch 6/40
1838283/1838283 [==============================] - 140s 76us/step - loss: 0.4633 - accuracy: 0.0000e+00 - val_loss: 0.6643 - val_acc

KeyboardInterrupt: 